## Documentation

To read more about the search by Query DSL, visit the docs [Aggregations](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations.html), [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-your-data.html) and [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html).

![search_aggregations_docs](../images/search_aggregations_docs.png)


## Connect to ElasticSearch


In [1]:
import { Client } from "npm:@elastic/elasticsearch";
import { load } from "https://deno.land/std/dotenv/mod.ts";

const env = await load({ envPath: "../.env" });

const client = new Client({
  node: env.ELASTICSEARCH_NODE,
  auth: {
    apiKey: env.ELASTICSEARCH_API_KEY,
  },
});

const info = await client.info();
console.log(info);


{
  name: "fead23d3120d",
  cluster_name: "docker-cluster",
  cluster_uuid: "v3fUyW9OReext6IjPiOCqg",
  version: {
    number: "8.17.4",
    build_flavor: "default",
    build_type: "docker",
    build_hash: "c63c7f5f8ce7d2e4805b7b3d842e7e792d84dda1",
    build_date: "2025-03-20T15:39:59.811110136Z",
    build_snapshot: false,
    lucene_version: "9.12.0",
    minimum_wire_compatibility_version: "7.17.0",
    minimum_index_compatibility_version: "7.0.0"
  },
  tagline: "You Know, for Search"
}


## Create index


In [4]:
await client.indices.delete({ index: "my_index", ignore_unavailable: true });
await client.indices.create({ index: "my_index" });


{ acknowledged: true, shards_acknowledged: true, index: "my_index" }

Let's index the documents.


In [5]:
import data from "../data/dummy_data_2.json" with { type: "json" };

function repeatArrayThousandTime(arr: any[]) {
  const result = [];
  for (let i = 0; i < 1000; i++) {
      result.push(...arr);
  }
  return result;
}

const dummyData = repeatArrayThousandTime(data);

console.log(dummyData.length);


5000


Since, we have duplicated the dummy data so much. Let's use the `bulk API` since we learned it before to index all those documents rapidly.


In [6]:
const operations = [];
for (const document of dummyData) {
  operations.push({
    index: {
      _index: "my_index",
    },
  });
  operations.push(document);
}

await client.bulk({
  operations,
});


{
  errors: false,
  took: 200,
  items: [
    {
      index: {
        _index: "my_index",
        _id: "RMi5AZYBo7TaDttMc2dx",
        _version: 1,
        result: "created",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 0,
        _primary_term: 1,
        status: 201
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "Rci5AZYBo7TaDttMc2dx",
        _version: 1,
        result: "created",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 1,
        _primary_term: 1,
        status: 201
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "Rsi5AZYBo7TaDttMc2dx",
        _version: 1,
        result: "created",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 2,
        _primary_term: 1,
        status: 201
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "R8i5AZYBo7TaDttMc2dx",
        _version: 1,
        result: "created",
       

## Aggregations

In this example, we perform an aggregation to calculate the average value of the `age` field across all documents that match the query. The result of the aggregation is stored in the `averageAge` key.


In [8]:
const response = await client.search({
  index: "my_index",
  query: {
    match_all: {},
  },
  aggs: {
    averageAge: {
      avg: {
        field: "age",
      },
    },
  },
});

console.log(`Average age: ${response.aggregations.averageAge.value}`);


Average age: 33.4


### Combining size, from, timeout, and aggs

Here we combine multiple parameters: we limit the results to 5 documents (`size`), skip the first 20 documents (`from`), set a timeout of 5 seconds (`timeout`), and perform a maximum aggregation (`aggs`) on the `price` field. This demonstrates how to use multiple search parameters together.

In [11]:
const response = await client.search({
  index: "my_index",
  query: {
    match: {
      message: "yes, keyword included",
    },
  },
  aggs: {
    maxPrice: {
      max: {
        field: "price",
      },
    },
  },
  size: 5,
  from: 20,
  timeout: "5s",
});

for (const hit of response.hits.hits) {
  console.log(hit._source);
}

console.log(`Max price: ${response.aggregations.maxPrice.value}`);


{
  message: "Randomized result: yes, keyword included.",
  age: 38,
  price: 149.25
}
{
  message: "Randomized result: yes, keyword included.",
  age: 38,
  price: 149.25
}
{
  message: "Randomized result: yes, keyword included.",
  age: 38,
  price: 149.25
}
{
  message: "Randomized result: yes, keyword included.",
  age: 38,
  price: 149.25
}
{
  message: "Randomized result: yes, keyword included.",
  age: 38,
  price: 149.25
}
Max price: 199.89999389648438
